# Using QuantumGraph

The `quantumgraph` package is designed to help build quantum software using an approach that is different from the one you'll learn in textbooks. Here we'll look into the basics of how it works.

## Setting up

The first step is to install `quantumgraph`. You'll also need to install the `pairwise_tomography` package, which is a vital dependency.

This can be done by pip installing directly from the repositories. This is usually done with the command line, but you can do it in Jupyter notebooks too. The syntax for Jupyter notebooks is as follows.

!pip install git+https://github.com/moth-quantum/QuantumGraph.git

With these things installed, you will need to import the class in which all the magic happens.

In [1]:
from quantumgraph import QuantumGraph

## Initializing the variables

Computing is all about encoding information in variables, and then manipulating these variables. So to start we need to understand what variables we will use and how to set them up.

The variables in `QuantumGraph` are associated with a set of qubits. We begin by creating a `QuantumGraph` object to hold our qubits. In the example below, we do it for a set of four qubits.

In [2]:
n = 4
qubits = QuantumGraph(n)

For each qubit we have three variables, $\langle X \rangle$ , $\langle Y \rangle$ and $\langle Z \rangle$. These are all numbers in the range between $+1$ and $-1$, and they satisfy

$$
\langle X \rangle^2 + \langle Y \rangle^2 + \langle Z \rangle^2 \leq 1.
$$

These values for a qubit `j` can be accessed as a dictionary using the `get_bloch()` method. Here it is in use.

In [3]:
for j in range(n):
    print('Single qubit variables for qubit ' + str(j) + ':')
    print(qubits.get_bloch(j))
    print()

Single qubit variables for qubit 0:
{'X': np.float64(0.010862931771024044), 'Y': np.float64(0.013548150860490663), 'Z': np.float64(1.0)}

Single qubit variables for qubit 1:
{'X': np.float64(-0.014524594165751251), 'Y': np.float64(0.02038325399731478), 'Z': np.float64(1.0)}

Single qubit variables for qubit 2:
{'X': np.float64(-0.012815818381545222), 'Y': np.float64(0.006957158549981692), 'Z': np.float64(1.0)}

Single qubit variables for qubit 3:
{'X': np.float64(0.004271939460515074), 'Y': np.float64(-0.009154155986818015), 'Z': np.float64(1.0)}



For each possible pair of qubits $j$ and $k$, we have a whole bunch of variables: $\langle X_j X_k \rangle$, $\langle X_j Y_k \rangle$, $\langle X_j Z_k \rangle$, $\langle Y_j X_k \rangle$, $\langle Y_j Y_k \rangle$, etc. For a fairly in-depth explation of these, see [this paper](https://arxiv.org/abs/2005.10327).

For now it is sufficient to note these variables are initially completely dependent on the single qubit variables. For example

$$
\langle X_j Y_k \rangle = \langle X_j \rangle \langle Y_k \rangle.
$$

They can begin to do different things as a computation proceeds, but they'll always satisfy

$$
\langle X_j Y_k \rangle^2 \geq \left( \langle X_j \rangle \langle Y_k \rangle \right)^2
$$

These values for a pair of qubits `j` and `k` can be similarly accessed using the `get_relationship()` method.

In [4]:
for j in range(n-1):
    for k in range(j+1,n):
        print('Two qubit variables for qubits ' + str(j) + ' and '  + str(k) + ':')
        print(qubits.get_relationship(j,k))
        print()

Two qubit variables for qubits 0 and 1:
{'XX': np.float64(-0.002807274502624191), 'XY': np.float64(0.007201269376296839), 'XZ': np.float64(-0.01720981325521787), 'YX': np.float64(-0.015256926644696692), 'YY': np.float64(0.004516050286830221), 'YZ': np.float64(-0.004271939460515074), 'ZX': np.float64(0.016721591602587575), 'ZY': np.float64(-0.012327596728914928), 'ZZ': np.float64(1.0)}

Two qubit variables for qubits 0 and 2:
{'XX': np.float64(0.009642377639448309), 'XY': np.float64(0.00207494202367875), 'XZ': np.float64(0.022580251434151103), 'YX': np.float64(0.004271939460515074), 'YY': np.float64(-0.008910045160502868), 'YZ': np.float64(-0.004516050286830221), 'ZX': np.float64(0.010862931771024044), 'ZY': np.float64(-0.015256926644696692), 'ZZ': np.float64(1.0)}

Two qubit variables for qubits 0 and 3:
{'XX': np.float64(-0.00012205541315757354), 'XY': np.float64(0.002563163676309044), 'XZ': np.float64(0.004271939460515074), 'YX': np.float64(0.004271939460515074), 'YY': np.float64(-0.

The results we see above show us that we are in the standard $|000\rangle$ initial state, for which

$$\langle Z_j \rangle = \langle Z_j Z_k \rangle = 1 \,\,\,\, \forall j,k$$

All variables involving $X$ and $Y$ are zero. Any deviations from this seen above are due to the fact that the expectation values are statistically calculated.

## Manipulating qubits

The qubits can be manipulated using normal quantum gates. For this we can directly manipulate the quantum circuit for the qubits, which can be accessed through the attribute `qc`. Since `QuantumGraph` is built on Qiskit, this circuit is a Qiskit `QuantumCircuit` object.

In [5]:
qubits.qc.h(0)
qubits.qc.draw()

┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────
          
q_2: ─────
          
q_3: ─────

Here we applied an `h` gate to qubit 0. However, if we look at the state of this qubit we will find that it has not changed.

In [6]:
qubits.get_bloch(0)

{'X': np.float64(0.010862931771024044),
 'Y': np.float64(0.013548150860490663),
 'Z': np.float64(1.0)}

This is because we actually have to run the circuit in order to update the tomography. This is done using the `update_tomography()` method.

In [7]:
qubits.update_tomography()
print(qubits.get_bloch(0))

{'X': np.float64(1.0), 'Y': np.float64(-0.002807274502624191), 'Z': np.float64(-0.008910045160502868)}


Now we have $\langle X \rangle = 1$ for qubit 0. If you know what an `h` does, this shouldn't come as a surprise (if you didn't know, then now do!).

Another way to manipulate the qubits, which is unique to this method, is to give target values for the variables. This is done using the method `set_bloch()`.

For example, to set the state of qubit 1 to $\langle X \rangle = 1$ we use

In [8]:
qubits.set_bloch({'X':1},1)

This automatically updates the tomography (unless the keyword `update=False` is added), so we can already verify that it has the correct effect.

In [9]:
print(qubits.get_bloch(1))

{'X': np.float64(0.9997558891736849), 'Y': np.float64(-0.004760161113145368), 'Z': np.float64(0.012571707555230075)}


The effect was acheived by considering the initial and target tomography, and finding a unitary to get as close to the target as possible.

In [10]:
qubits.qc.draw()

┌───┐               
q_0: ───────────────┤ H ├───────────────
     ┌──────────────┴───┴──────────────┐
q_1: ┤ U(1.5734,-0.011351,-1.4546e-05) ├
     └─────────────────────────────────┘
q_2: ───────────────────────────────────
                                        
q_3: ───────────────────────────────────

It is not as pretty as the off-the-shelf Hadamard gate, but it has the same effect.

The target state for the `set_bloch()` method can include multiple values. For example


In [11]:
qubits.set_bloch({'X':1, 'Z':1},2)

print(qubits.get_bloch(2))

{'X': np.float64(0.7117051141218113), 'Y': np.float64(0.044062004149884045), 'Z': np.float64(0.7019406810692054)}


Note that it is not actually possible to set both $\langle X \rangle$ and $\langle Z \rangle$ of a qubit to $1$, so `set_bloch` just does the best that is possible. For all single qubit values that do not appear in the target state, it is assumed that the target is to set them to zero. So the target used here is equivalent to `{'X':1, 'Y':0, 'Z':1,}`.


We can perform similar manipulations on pairs of qubits using the `set_relationship` method. Here we do so for qubits 0 and 1.

In [12]:
qubits.set_relationship( {'ZZ':1}, 0,1 )

print(qubits.get_relationship(0,1))

{'XX': np.float64(1.0), 'XY': np.float64(0.0013426095447333088), 'XZ': np.float64(-0.008421823507872574), 'YX': np.float64(-0.016477480776272427), 'YY': np.float64(-1.0), 'YZ': np.float64(-0.016477480776272427), 'ZX': np.float64(0.0008543878921030148), 'ZY': np.float64(0.008421823507872574), 'ZZ': np.float64(0.9992676675210546)}


Unlike `set_bloch`, it is not assumed that values not present in the target should be set to zero. Instead, it is assumed that they should be changed as little as possible.

In the example above, the target is to set $\langle Z_0 Z_1 \rangle = 1$. Note that the state already has $\langle X_0 \rangle = \langle X_0 \rangle = 1$, and therefore $\langle X_0 X_1 \rangle = 1$, from the single qubit manipulations we have already performed. Since $Z_0 Z_1$ and $X_0 X_1$ commute, it is possible to satisfy the target condition on $\langle Z_0 Z_1 \rangle$ while maintaining the value of $\langle X_0 X_1 \rangle$. It is because of this that we end up with a state with $\langle X_0 X_1 \rangle = \langle Z_0 Z_1 \rangle = 1$.

We can also use `set_relationship()` with multiple values in the target. However, unlike for `set_bloch()`, all the corresponding Paulis must commute to get a well defined effect. For example, for qubits 2 and 3.

In [13]:
qubits.set_relationship( {'XZ':1,'ZX':1}, 2,3 )

print(qubits.get_relationship(2,3))

{'XX': np.float64(0.008421823507872574), 'XY': np.float64(0.006468936897351398), 'XZ': np.float64(0.9997558891736849), 'YX': np.float64(0.02892713291834493), 'YY': np.float64(0.9997558891736849), 'YZ': np.float64(0.004271939460515074), 'ZX': np.float64(0.9997558891736849), 'ZY': np.float64(-0.001830831197363603), 'ZZ': np.float64(-0.01305992920786037)}


Both `set_bloch` and `set_relationship` also have a `fraction` keyword argument. If we use $U$ to denote the unitary that would have been applied normally, setting a value for `fraction` applies $U^{\mathtt{fraction}}$ instead.

Let's get a fresh pair of qubits to see an example of this.

In [14]:
qubits = QuantumGraph(2)

print(qubits.get_relationship(0,1))

{'XX': np.float64(-0.001830831197363603), 'XY': np.float64(0.00793360185524228), 'XZ': np.float64(-0.001830831197363603), 'YX': np.float64(0.002563163676309044), 'YY': np.float64(-0.009642377639448309), 'YZ': np.float64(-0.0032954961552544857), 'ZX': np.float64(0.00207494202367875), 'ZY': np.float64(-0.0035396069815696327), 'ZZ': np.float64(1.0)}


Applying `qubits.set_relationship({'XX':1}, 0,1)` would change the state to get $\langle X_0 X_1 \rangle = 1$. But let's add in `fraction=0.5` and see what happens.

In [15]:
qubits.set_relationship( {'XX':1}, 0,1, fraction=0.5 )

print(qubits.get_relationship(0,1))

{'XX': np.float64(0.6936409129744904), 'XY': np.float64(-0.2876846088124008), 'XZ': np.float64(-0.020871475649945074), 'YX': np.float64(-0.29207860368607347), 'YY': np.float64(-0.6518979616746002), 'YZ': np.float64(-0.15000610277065787), 'ZX': np.float64(0.07213474917612596), 'ZY': np.float64(-0.17929940192847552), 'ZZ': np.float64(0.965092151836934)}


Here the value has increased $1/\sqrt{2}$, but not all the way to $1$. In some sense, it is half way there.

If we were to apply exactly the same quantum gate operation again, we would get all the way to $\langle X_0 X_1 \rangle = 1$. However, note that we cannot do this by simply repeating the same command, since this would generate a new gate which rotates halfway between the current initial state and the target state.

In [16]:
qubits.set_relationship( {'XX':1}, 0,1, fraction=0.5 )

print(qubits.get_relationship(0,1))

{'XX': np.float64(0.9167582082265349), 'XY': np.float64(-0.16196753326010008), 'XZ': np.float64(-0.02892713291834493), 'YX': np.float64(-0.156841205907482), 'YY': np.float64(-0.8613450506529965), 'YZ': np.float64(-0.2403271085072623), 'ZX': np.float64(0.041620895886732574), 'ZY': np.float64(-0.23178322958623215), 'ZZ': np.float64(0.9528866105211766)}


# Creating Algorithms

By thinking of terms of these single- and two-qubit variables, and by manipulating them using the methods supplied, we can create algorithms to solve problems.

A concrete example of this can be seen in

* ["A quantum procedure for map generation", by James R. Wootton](https://arxiv.org/abs/2005.10327)

where `quantumgraph` is used to implement a rudimentary AI for a Civilization-like game.

Another example is

* ["Determining eigenstates and thermal states on a quantum computer using quantum imaginary time evolution" by Motta, et al.](https://arxiv.org/abs/1901.07653)

which is presents a method for calculating ground states for Hamiltonians of interacting particles. It uses the same basic idea as is implemented in `quantumgraph`, but with a $p$-local tomography in general (for finite $p$) and with its own custom methods to set the variables.